# Solving (Set) Constraints in B and Event-B #

## A quick Introduction to B ##

### Basic Datavalues in B ###

B provides the booleans, strings and integers as built-in datatypes. (Strings are not available in Event-B.)

In [1]:
BOOL

{FALSE,TRUE}

In [2]:
"this is a string"

"this is a string"

In [3]:
1024

1024

Users can define their own datatype in a B machine.
One distinguishes between explicitly specified enumerated sets and deferred sets.

In [14]:
::load
MACHINE MyBasicSets
SETS Trains = {thomas, gordon}; Points
END

[2018-06-04 17:10:21,614, T+7877565] "Shell-0" de.prob.cli.PrologProcessProvider.makeProcess(PrologProcessProvider.java:64): [INFO] Starting ProB's Prolog Core. Path is /Users/leuschel/git_root/prob_prolog/probcli.sh
[2018-06-04 17:10:22,773, T+7878724] "Shell-0" de.prob.cli.PortPattern.setValue(PortPattern.java:30): [INFO] Server has started and listens on port 57165
[2018-06-04 17:10:22,774, T+7878725] "Shell-0" de.prob.cli.InterruptRefPattern.setValue(InterruptRefPattern.java:29): [INFO] Server can receive user interrupts via reference 52908
[2018-06-04 17:10:22,782, T+7878733] "ProB Output Logger for instance 17752aaa" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] -- starting command loop --
[2018-06-04 17:10:22,792, T+7878743] "ProB Output Logger for instance 17752aaa" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Connected: 127.0.0.1


Loaded machine: MyBasicSets : []


In [5]:
Trains

{thomas,gordon}

For animation and constraint solving purposes, ProB will instantiate deferred sets to some finite set (the size of which can be controlled and is partially inferred).

In [6]:
Points

{Points1,Points2}

### Pairs ###
B also has pairs of values, which can be written in two ways:

In [7]:
(thomas,10)

(thomas↦10)

In [8]:
thomas |-> 10

(thomas↦10)

Tuples simply correspond to nested pairs:

In [9]:
(thomas |-> gordon |-> 20)

((thomas↦gordon)↦20)

Classical B also provides records:

In [15]:
rec(train:thomas,length:20,position:10302)

rec(length∈20,position∈10302,train∈thomas)

### Sets ###
Sets in B can be specified in multiple ways.
For example, using explicit enumeration:

In [10]:
{1,3,2,3}

{1,2,3}

or via a predicate by using a set comprehension:

In [11]:
{x|x>0 & x<4}

{1,2,3}

One can use on of the *base sets* :

In [12]:
(BOOL,INTEGER,STRING,Trains,Points)

(((({FALSE,TRUE}↦INTEGER)↦STRING)↦{thomas,gordon})↦{Points1,Points2})

For integers there are a variety of other sets, such as intervals:

In [13]:
1..3

{1,2,3}

or the set of implementable integers INT = MININT..MAXINT or the set of implementable natural numbers NAT = 0..MAXINT.

Sets can be higher-order and contain other sets:

In [14]:
{ 1..3,  {1,2,3,2}, 0..1, {x|x>0 & x<4} }

{{0,1},{1,2,3}}

Relations are modelled as sets of pairs:

In [16]:
{ thomas|->gordon, gordon|->gordon, thomas|->thomas}

{(thomas↦thomas),(thomas↦gordon),(gordon↦gordon)}

Note: a pair is an element of a Cartesian product, and a relation is just a subset of a Cartesian product.
The above relation is a subset of:

In [17]:
Trains * Trains

{(thomas↦thomas),(thomas↦gordon),(gordon↦thomas),(gordon↦gordon)}

Functions are relations which map every domain element to at most one value:

In [18]:
{ thomas|->1, gordon|->2}

{(thomas↦1),(gordon↦2)}

## Expressions vs Predicates vs Substitutions ##


### Expressions ###
Expressions in B have a value. With ProB and with ProB's Jupyter backend, you can evaluate expresssions such as:

In [19]:
2**1000

10715086071862673209484250490600018105614048117055336074437503883703510511249361224931983788156958581275946729175531468251871452856923140435984577574698574803934567774824230985421074605062371141877954182153046474983581941267398767559165543946077062914571196477686542167660429831652624386837205668069376

B provides many operators which return values, such as the usual arithmetic operators but also many operators for sets, relations and functions.
For example set union and set difference:

In [20]:
(1..3 \/ 5..10) \ (2..6)

{1,7,8,9,10}

In [21]:
(1..3 ∪ 5..10) \ (2..6)

{1,7,8,9,10}

The range of a relation or function:

In [22]:
ran({(thomas↦1),(gordon↦2)})

{1,2}

Function application:

In [ ]:
{(thomas↦1),(gordon↦2)} (thomas)

Relational inverse (.~) and relational image .[.] :

In [ ]:
{(thomas↦1),(gordon↦2)}~[2..3]

## Predicates
ProB can also be used to evaluate predicates (B distinguishes between expressions which have a value and predicates which are either true or false).

In [23]:
2>3

FALSE

In [24]:
3>2

TRUE

Within predicates you can use **open** variables, which are implicitly existentially quantified.
ProB will display the solution for the open variables, if possible.

In [25]:
x*x=100

TRUE

Solution:
	x = −10

We can find all solutions to a predicate by using the set comprehension notation.
Note that by this we turn a predicate into an expression.

In [26]:
{x|x*x=100}

{−10,10}

### Substitutions ###
B also has a rich syntax for substitutions, aka statements.
For example ```x := x+1``` increments the value of x by 1.
We will not talk about substitutions in the rest of this presentation.
The major differences between classical B and Event-B lie in the area of substitutions, machine composition and refinement.

## Definition of Constraint Solving ##

Constraint solving is determine whether a predicate with open/existentially quantified variables is satisfiable and providing values for the open variables in case it is.
We have already solved the predicate ```x*x=100``` above, yielding the solution ```x=-10```.
The following is an unsatisfiable predicate:

In [27]:
x*x=1000

FALSE

The difference to **proof** is that in constraint solving one has to produce a solution (aka a model). The difference to **execution** is that not all variables are known.

## Constraint Solving Applications ##
Constraint solving has many applications in formal methods in general and B in particular.



#### Animation ####
It is required to animate implicit specifications.
Take for example an event
```
train_catches_up = any t1,t2,x where t1:dom(train_position) & t2:dom(train_position) &
                                     train_position(t1) < train_position(t2) &
                                     x:1..(train_position(t2)-train_position(t1)-1) then
                         train_position(t1) := train_position(t1)+x end
```
To determine whether the event is enabled, and to obtain values for the parameters of the event in a given state of the model, we have to solve the following constraint:

In [28]:
train_position = {thomas|->100, gordon|->2020} &
t1:dom(train_position) & t2:dom(train_position) & train_position(t1) < train_position(t2) &
x:1..(train_position(t2)-train_position(t1)-1)

TRUE

Solution:
	x = 1
	train_position = {(thomas↦100),(gordon↦2020)}
	t1 = thomas
	t2 = gordon

In [29]:
train_position = {thomas|->2019, gordon|->2020} &
t1:dom(train_position) & t2:dom(train_position) & train_position(t1) < train_position(t2) &
x:1..(train_position(t2)-train_position(t1)-1)

FALSE

#### Feasibility Analysis ####
Suppose that we have the invariant, ```train_position:TRAINS-->1..10000``` we can check whether the event is **feasible** in at least one valid state by solving:

In [30]:
train_position:Trains-->1..10000 &
t1:dom(train_position) & t2:dom(train_position) & train_position(t1) < train_position(t2) &
x:1..(train_position(t2)-train_position(t1)-1)

TRUE

Solution:
	x = 1
	train_position = {(thomas↦1),(gordon↦3)}
	t1 = thomas
	t2 = gordon

Many other applications exist: generating **testcases**, finding counter examples using **bounded model checking** or other algorithms like IC3.
Other applications are analysing **proof obligations**.
Take the proof obligation for an event theorem t1 $/=$ t2:
```
train_position:Trains-->1..10000 & train_position(t1) < train_position(t2) |- t1 /= t2
```
We can find counter examples to it by negating the proof goal:

In [31]:
train_position:Trains-->1..10000 & train_position(t1) < train_position(t2) & not( t1 /= t2 )

FALSE

As no counter example has been found, we can in this case establish the goal to be proven.

#### Modelling and Solving Problems in B ####

Obviously, we can also use constraint solving to solve puzzles or real-life problems.
#### Send More Money Puzzle ####
We now try and solve the SEND+MORE=MONEY arithmetic puzzle in B, involving 8 distinct digits:

In [32]:
:prettyprint {S,E,N,D, M,O,R, Y} <: 0..9 & S >0 & M >0 & card({S,E,N,D, M,O,R, Y}) = 8 & 
   S*1000 + E*100 + N*10 + D + M*1000 + O*100 + R*10 + E = M*10000 + O*1000 + N*100 + E*10 + Y

{S,E,N,D,M,O,R,Y} ⊆ 0 ‥ 9 ∧ S > 0 ∧ M > 0 ∧ card({S,E,N,D,M,O,R,Y}) = 8 ∧ S * 1000 + E * 100 + N * 10 + D + M * 1000 + O * 100 + R * 10 + E = M * 10000 + O * 1000 + N * 100 + E * 10 + Y

In [16]:
{S,E,N,D, M,O,R, Y} ⊆ 0..9 & S >0 & M >0 & 
   card({S,E,N,D, M,O,R, Y}) = 8 & 
   S*1000 + E*100 + N*10 + D +
   M*1000 + O*100 + R*10 + E =
  M*10000 + O*1000 + N*100 + E*10 + Y

TRUE

Solution:
	R = 8
	S = 9
	D = 7
	E = 5
	Y = 2
	M = 1
	N = 6
	O = 0

We can find all solutions (to the unmodified puzzle) using a set comprehension and make sure that there is just a single soltuion:

In [34]:
  {S,E,N,D, M,O,R, Y |
   {S,E,N,D, M,O,R, Y} <: 0..9 &  S >0 & M >0 & 
   card({S,E,N,D, M,O,R, Y}) = 8 & 
   S*1000 + E*100 + N*10 + D +
   M*1000 + O*100 + R*10 + E =
   M*10000 + O*1000 + N*100 + E*10 + Y }

{(((((((9↦5)↦6)↦7)↦1)↦0)↦8)↦2)}

In [15]:
:table   {S,E,N,D, M,O,R, Y |
   {S,E,N,D, M,O,R, Y} <: 0..9 &  S >0 & M >0 & 
   card({S,E,N,D, M,O,R, Y}) = 8 & 
   S*1000 + E*100 + N*10 + D +
   M*1000 + O*100 + R*10 + E =
   M*10000 + O*1000 + N*100 + E*10 + Y }

|Nr|S|E|N|D|M|O|R|Y|
|---|---|---|---|---|---|---|---|---|
|1|9|5|6|7|1|0|8|2|


#### KISS PASSION Puzzle####
A slightly more complicated puzzle (involving multiplication) is the KISS * KISS = PASSION problem.

In [35]:
   {K,P} <: 1..9 &
    {I,S,A,O,N} <: 0..9 &
    (1000*K+100*I+10*S+S) * (1000*K+100*I+10*S+S) 
     =  1000000*P+100000*A+10000*S+1000*S+100*I+10*O+N &
    card({K, I, S, P, A, O, N}) = 7

TRUE

Solution:
	P = 4
	A = 1
	S = 3
	I = 0
	K = 2
	N = 9
	O = 8

Finally, a simple puzzle involving sets is to find a subset of numbers from 1..5 whose sum is 14:

In [36]:
x <: 1..5 & SIGMA(y).(y:x|y)=14

TRUE

Solution:
	x = {2,3,4,5}

## How to solve (set) constraints in B ##

We will now examine how one can perform constraint solving for B.

### Booleans ###

If we have only booleans, constraint solving is equivalent to SAT solving.
Internally, ProB has an interpreter which does **not** translate to CNF (conjunctive normal form), but is otherwise similar to DPLL: deterministic propagations are carried out first (unit propagation) and there are heuristics to choose the next boolean variable to enumerate.
(We do not translate to CNF also because of well-definedness issues.)



#### Knights and Knave Puzzle####
Here is a puzzle from Smullyan involving an island with only knights and knaves.
We know that:
 - Knights: always tell the truth
 - Knaves: always lie

We are given the following information about three persons A,B,C on the island:
 1. A says: “B is a knave or C is a knave”
 2. B says “A is a knight”

What are A, B and C?
Note: we model A,B,C as boolean variables which are equal to TRUE if they are a knight and FALSE if they are a knave.

In [37]:
 (A=TRUE <=> (B=FALSE or C=FALSE)) & // Sentence 1
 (B=TRUE <=> A=TRUE) // Sentence 2

TRUE

Solution:
	A = TRUE
	B = TRUE
	C = FALSE

### Integers ###

Let us take the integer constraint ```x*x=100``` which we saw earlier.
This constraint is actually more complicated than might appear at first sight: the constraint is not linear and the domain of x is not bounded. Indeed, B supports mathematical integers without any bit size restriction.
So, let us first look at some simpler constraints, where the domains of the variables are all bounded.

Let us look at the simple constraint ```X:0..3 & Y:0..3 & X+Y=2```.
As you can see there are three solutions for this constraint:


In [39]:
{X,Y|X:0..3 & Y:0..3 & X+Y=2}

{(0↦2),(1↦1),(2↦0)}

We will now study how such constraints can be solved.

#### Solving constraints by translating to SAT ####
Given that we know the *domain* of X and Y in the constraint ``` X:0..3 & Y:0..3 & X+Y=2```, we can represent the integers by binary numbers and convert the constraint to a **SAT** problem.
The number 2 is ```10``` in binary and we can represent X and Y each by two bits X0,X1 and Y0,Y1.
We can translate the addition to a propositional logic formula:

Bit1 | Bit0
-----|-----
   X1  |  X0
 + Y1  |  Y0
     |
 Z1  |  Z0
 
Let us find one solution to this constraint, by encoding addition using an additional carry bit ```CARRY0```:

In [40]:
 ((X0=TRUE <=> Y0=TRUE) <=> Z0=FALSE) & 
 ((X0=TRUE & Y0=TRUE) <=> CARRY0=TRUE) & 
 (CARRY0=FALSE => ((X1=TRUE <=> Y1=TRUE) <=> Z1=FALSE)) &
 (CARRY0=TRUE => ((X1=TRUE <=> Y1=TRUE) <=> Z1=TRUE)) &
 Z0=FALSE & Z1=TRUE

TRUE

Solution:
	Z0 = FALSE
	Y0 = TRUE
	Z1 = TRUE
	X0 = TRUE
	Y1 = TRUE
	X1 = TRUE
	CARRY0 = TRUE

In [41]:
{X0,X1,Y0,Y1,Z0,Z1,CARRY0 | ((X0=TRUE <=> Y0=TRUE) <=> Z0=FALSE) & 
   ((X0=TRUE & Y0=TRUE) <=> CARRY0=TRUE) & 
   (CARRY0=FALSE => ((X1=TRUE <=> Y1=TRUE) <=> Z1=FALSE)) & 
   (CARRY0=TRUE => ((X1=TRUE <=> Y1=TRUE) <=> Z1=TRUE)) &
    Z0=FALSE & Z1=TRUE}

{((((((FALSE↦FALSE)↦FALSE)↦TRUE)↦FALSE)↦TRUE)↦FALSE),((((((FALSE↦TRUE)↦FALSE)↦FALSE)↦FALSE)↦TRUE)↦FALSE),((((((TRUE↦FALSE)↦TRUE)↦FALSE)↦FALSE)↦TRUE)↦TRUE),((((((TRUE↦TRUE)↦TRUE)↦TRUE)↦FALSE)↦TRUE)↦TRUE)}

As you can see, we have found four solutions and not three! One solution is 3+3=2.
This is a typical issue when translating arithmetic to binary numbers: we have to prevent overflows, which we do below:

In [42]:
{X0,X1,Y0,Y1,Z0,Z1,CARRY0 | ((X0=TRUE <=> Y0=TRUE) <=> Z0=FALSE) & 
   ((X0=TRUE & Y0=TRUE) <=> CARRY0=TRUE) & 
   (CARRY0=FALSE => ((X1=TRUE <=> Y1=TRUE) <=> Z1=FALSE)) & 
   (CARRY0=TRUE => ((X1=TRUE <=> Y1=TRUE) <=> Z1=TRUE)) &
   (CARRY0=TRUE => (X1=FALSE & Y1=FALSE)) & // no overflow
   (CARRY0=FALSE => (X1=FALSE or Y1=FALSE)) & // no overflow
    Z0=FALSE & Z1=TRUE}

{((((((FALSE↦FALSE)↦FALSE)↦TRUE)↦FALSE)↦TRUE)↦FALSE),((((((FALSE↦TRUE)↦FALSE)↦FALSE)↦FALSE)↦TRUE)↦FALSE),((((((TRUE↦FALSE)↦TRUE)↦FALSE)↦FALSE)↦TRUE)↦TRUE)}

In ProB, we can use **Kodkod** backend to achieve such a translation to SAT.
- Kodkod (https://github.com/emina/kodkod) is the API to the **Alloy** (http://alloytools.org) constraint analyzer and takes relational logic predicates and translates them to SAT.
- The SAT problem can be solved by any SAT solver (Sat4J, minisat, glucose,...).
- ProB translates parts of B to the Kodkod API and translates the results back to B values.
- Prior to the translation, ProB performs an interval analysis to determine possible ranges for the integer decision variables.

The details were presented at FM'2012 (Plagge, L.).

![ProBKodkod](./img/ProB_Kodkod_Architecture.png)

In [43]:
:solve kodkod x:0..2 & y:0..2 & x+y=2

[2018-06-04 14:01:28,941, T+458790] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   x : 0 .. 2 & y : 0 .. 2 & x + y = 2  ints: irange(0,4), intatoms: none
[2018-06-04 14:01:28,941, T+458790] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Kodkod module started up successfully (SAT solver SAT4J with timeout of 1500 ms).
[2018-06-04 14:01:28,942, T+458791] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Times for computing solutions: [2,5,2]


TRUE

Solution:
	x = 2
	y = 0

We can find all solutions and check that we find exactly the three expected solutions:

In [44]:
:solve kodkod {x,y|x:0..2 & y:0..2 & x+y=2}=res

[2018-06-04 14:01:31,052, T+460901] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   {x,y|x : 0 .. 2 & y : 0 .. 2 & x + y = 2} = res  ints: irange(0,4), intatoms: irange(0,2)
[2018-06-04 14:01:31,054, T+460903] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Times for computing solutions: [1]


TRUE

Solution:
	res = {(0↦2),(1↦1),(2↦0)}

#### Translation to SMTLib ####
At iFM'2016 we (Krings, L.) presented a translation to SMTLib format to use either the **Z3** or **CVC4** SMT solver.
Compared to the Kodkod backend, no translation to SAT is performed, SMTLib supports **integer** predicates and integer operators in the language.

![ProBZ3](./img/inputoutputflow.pdf)


Here is ProB's SMTLib/Z3 API calls for the constraint ``` X:0..3 & Y:0..3 & X+Y=2```:
- mk_var(integer,x) $\rightarrow$ 2
- mk_var(integer,y) $\rightarrow$ 3
- mk_int_const(0) $\rightarrow$ 4
- mk_op(greater_equal,2,4) $\rightarrow$ 5
- mk_int_const(2) $\rightarrow$ 6
- mk_op(greater_equal,6,2) $\rightarrow$ 7
- mk_int_const(0) $\rightarrow$ 8
- mk_op(greater_equal,3,8) $\rightarrow$ 9
- mk_int_const(2) $\rightarrow$ 10
- mk_op(greater_equal,10,3) $\rightarrow$ 11
- mk_op(add,2,3) $\rightarrow$ 12
- mk_int_const(2) $\rightarrow$ 13
- mk_op(equal,12,13) $\rightarrow$ 14
- mk_op_arglist(conjunct,[5,7,9,11,14]) $\rightarrow$ 15

In [45]:
:solve z3 x:0..2 & y:0..2 & x+y=2

TRUE

Solution:
	x = 0
	y = 2

#### ProB's CLP(FD) Solver ####
ProB's default solver makes use of constraint logic programming.
For arithmetic, it builts on top of CLP(FD), the finite domain library of SICStus Prolog.
In CLP(FD):
- every integer variable is associated with a domain of possible values, typically an interval
- when adding a new constraints, the domains of the involved variables are updated, or more precisely narrowed down.
- at some point we need to chose variables for enumeration; typically ProB chooses the value with the smallest domain.

Let us use a slightly adapted constraint ```x:0..9 & y:0..9 & x+y=2``` to illustrate how constraint processing works:

- x:0..9 $\leadsto$ x:0..9, y:$-\infty$..$\infty$
- y:0..9 $\leadsto$ x:0..9, y:0..9
- x+y=2 $\leadsto$ x:0..2, y:0..2
- Enumerate (label) variable x
 - x=0 $\leadsto$ x:0..0, y:2..2
 - x=1 $\leadsto$ x:1..1, y:1..1
 - x=2 $\leadsto$ x:2..2, y:0..0

Let us now examine the three approaches for the KISS-PASSION puzzle:

In [17]:
:time :solve prob {K,P} <: 1..9 & {I,S,A,O,N} <: 0..9 & (1000*K+100*I+10*S+S) * (1000*K+100*I+10*S+S)  =  1000000*P+100000*A+10000*S+1000*S+100*I+10*O+N & card({K, I, S, P, A, O, N}) = 7

Execution time: 0.050491290 seconds

TRUE

Solution:
	P = 4
	A = 1
	S = 3
	I = 0
	K = 2
	N = 9
	O = 8

In [18]:
:time :solve z3 {K,P} <: 1..9 & {I,S,A,O,N} <: 0..9 & (1000*K+100*I+10*S+S) * (1000*K+100*I+10*S+S)  =  1000000*P+100000*A+10000*S+1000*S+100*I+10*O+N & card({K, I, S, P, A, O, N}) = 7

CommandExecutionException: :time: :solve: Computation not completed: no solution found (but one might exist)

In [19]:
:time :solve kodkod {K,P} <: 1..9 & {I,S,A,O,N} <: 0..9 & (1000*K+100*I+10*S+S) * (1000*K+100*I+10*S+S)  =  1000000*P+100000*A+10000*S+1000*S+100*I+10*O+N & card({K, I, S, P, A, O, N}) = 7

[2018-06-04 21:43:31,067, T+24267018] "ProB Output Logger for instance 17752aaa" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   K : 1 .. 9 & P : 1 .. 9 & I : 0 .. 9 & S : 0 .. 9 ...  ints: irange(0,99980001), intatoms: irange(0,9)
[2018-06-04 21:43:31,069, T+24267020] "ProB Output Logger for instance 17752aaa" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Kodkod module started up successfully (SAT solver SAT4J with timeout of 1500 ms).
[2018-06-04 21:43:31,069, T+24267020] "ProB Output Logger for instance 17752aaa" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Times for computing solutions: [636]


Execution time: 2.287290784 seconds

TRUE

Solution:
	P = 4
	A = 1
	S = 3
	I = 0
	K = 2
	N = 9
	O = 8

Result for KISS*KISS=PASSION puzzle:

Solver | Runtime
-------|-------
ProB Default | 0.01 sec
Kodkod Backend | 1 sec
Z3 Backend | ? > 100 sec

### Unbounded integers ###
The SAT translation via Kodkod/Alloy requires to determine the bid width.
It cannot be applied to unbounded integers.
Even for bounded integers it is quite tricky to get the bid widths correct: one needs also to take care of intermediate results. Alloy can detect incorrect models where an overflow occured, but to our understanding not where an overflow prevented a model (e.g., use inside negation or equivalence, see ```#(V.SS->V.SS)=0 iff no V.SS``` in paper at ABZ conference).

SMTLib is more tailored towards proof than towards model finding; as such it has typically no/less issues with unbounded values.
The ProB default solver can also deal with unbounded integers: it tries to narrow down domains to finite ones. If this fails, an unbounded variable is enumerated (partially) and an **enumeration warning** is generated. In case a solution is found, this warning is ignored, otherwise the result of ProB's analysis is **UNKNOWN**.
Some inconsistencies cannot be detected by interval/domain propagation; here it helps to activate ProB's CHR module which performs some additional inferences.

Let us perform some experiments. Both ProB and Z3 can solve the following:

In [49]:
:solve z3 x*x=100

TRUE

Solution:
	x = −10

Here is an example where ProB generates an enumeration warning, but finds a solution:

In [50]:
x>100 & x mod 2000 = 1 & x mod 3000 = 1

[2018-06-04 14:01:46,410, T+476259] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] ### Warning: enumerating x : INTEGER : 6001:sup ---> 6001:6001


TRUE

Solution:
	x = 6001

In [51]:
:solve z3 x>100 & x mod 2000 = 1 & x mod 3000 = 1

TRUE

Solution:
	x = 6001

Here ProB generates an enumeration warning and does not find a solution, hence the result is **UNKNOWN**. Here Z3 finds a solution.

In [52]:
:solve prob x>100 & x mod 2000 = 1 & x mod 3000 = 1 & (x+x) mod 4501 = 0

[2018-06-04 14:01:48,945, T+478794] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] ### Warning: enumerating x : INTEGER : 6001:sup ---> 6001:6001


CommandExecutionException: :solve: Computation not completed: no solution found (but one might exist)

In [53]:
:solve z3 x>100 & x mod 2000 = 1 & x mod 3000 = 1 & (x+x) mod 4501 = 0

TRUE

Solution:
	x = 6756001

Here is an inconsistency which cannot be detected by CLP(FD)'s interval propagation.
ProB can detect it with CHR (Constraint Handling Rules) enabled, but without the module the result is **UNKNOWN**.

In [54]:
:solve z3 x>y & y>x

FALSE

In [55]:
:solve prob x>y &y>x

[2018-06-04 14:01:54,055, T+483904] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] % Timeout when posting constraint:
[2018-06-04 14:01:54,055, T+483904] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] % kernel_objects:(_2222183#>0)
[2018-06-04 14:01:54,056, T+483905] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] ### Warning: enumerating x : INTEGER : inf:sup ---> -1:3


CommandExecutionException: :solve: Computation not completed: no solution found (but one might exist)

### Summary for Integer Arithmetic ###

Solver | Unbounded | Model Finding | Inconsistency Detection (Unbounded)
------|------------|---------------|---
ProB CLP(FD) | yes | very good | limited with CHR
ProB Z3 | yes | reasonable | very good
ProB Kodkod | no | good | -


### Deferred and Enumerated Sets ###

Given an enumerated set ```Trains = {thomas, gordon}``` we associate a variable ```x:Trains``` with
an integer decision variable in the range 1..2.
Similarly, deferred sets are given a finite cardinality $n$, and a decision variables are in the range 1..n.

In [56]:
x:Trains & y:Trains & x/=y

TRUE

Solution:
	x = thomas
	y = gordon

## Set Constraints ##

After booleans, integers and enumerated set elements, let us now move to constraint solving involving set variables.

### Translation to SAT ###
The Kodkod/Alloy backend translates sets bit vectors. The size of the vector is the number of possible elements.

Take for example the following constraint:

In [57]:
x <: 1..2 & y <: 1..2 & x \/ y = 1..2 & 1:x & x <<: y

TRUE

Solution:
	x = {1}
	y = {1,2}

In [58]:
{x1,x2,y1,y2} <: BOOL &
 x1=TRUE or y1=TRUE & x2=TRUE or y2=TRUE &   // x \/ y = 1..2
 x1=TRUE &   // 1:x 
 (x1=TRUE => y1=TRUE) & (x2=TRUE => y2=TRUE) & (x1/=y1 or x2/=y2)  // x <<: y

TRUE

Solution:
	y1 = TRUE
	x1 = TRUE
	y2 = TRUE
	x2 = FALSE

This translation to SAT is exactly what the Kodkod backend does:

In [59]:
:solve kodkod x <: 1..2 & y<: 1..2 & x \/ y = 1..2 & 1:x & x <<: y

[2018-06-04 14:02:32,427, T+522276] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   x <: 1 .. 2 & y <: 1 .. 2 & x \/ y = 1 .. 2 & 1 : ...  ints: irange(1,2), intatoms: irange(1,2)
[2018-06-04 14:02:32,428, T+522277] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Times for computing solutions: [1]


TRUE

Solution:
	x = {1}
	y = {1,2}

Limitations of translating set constraints to SAT:
- this cannot deal with **unbounded** sets: we need to know a finite type for each set, so that we can generate a finite bit vector
- this approach cannot usually deal with **higher order** sets (sets of sets), as the size of the bit vector would be prohibitively large

Given that:

In [60]:
card(POW(1..100))

1267650600228229401496703205376

translating the following constraint to SAT would require a bit vector of length 1267650600228229401496703205376.

In [61]:
x <: POW(1..100) & {100}:x & !y.(y:x => {card(y)}:x)

TRUE

Solution:
	x = {{100},{1}}

Also, in the following constraint, the set x is unbounded and no translation to SAT is feasible (without a very clever analysis of the universal implications).

In [62]:
{100}:x & !y.(y:x => (!z.(z:y => y \/ {z / 2}:x)))

TRUE

Solution:
	x = {{100},{50,100},{25,50,100},{12,25,50,100},{6,12,25,50,100},{3,6,12,25,50,100},{1,3,6,12,25,50,100},{0,1,3,6,12,25,50,100}}

However, when it is applicable the propositional encoding of sets can be very effective for SAT solvers.
The following example, involving various relational operators can currently only be solved by our Kodkod backend:

In [20]:
:time :solve kodkod r: 1..5 <-> 1..5 & (r;r) = r & r /= {} & dom(r)=1..5 & r[2..3]=4..5

[2018-06-04 21:54:17,800, T+24913751] "ProB Output Logger for instance 17752aaa" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   r : 1 .. 5 <-> 1 .. 5 & (r ; r) = r & r /= {} & do...  ints: irange(0,5), intatoms: irange(0,5)
[2018-06-04 21:54:17,801, T+24913752] "ProB Output Logger for instance 17752aaa" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Times for computing solutions: [2,2,1,1,3,2,1,2,1,1,5,1,1,2,1,2,1,2,1,1,1,1]


Execution time: 0.153314218 seconds

TRUE

Solution:
	r = {(3↦5),(3↦4),(5↦5),(5↦4),(1↦4),(2↦4),(4↦4)}

### Translation to SMTLib ###

This can in principle deal with higher-order sets and unbounded sets, but makes heavy use of quantifiers.
The practical usefulness is currently very limited.

In [63]:
:solve z3 x ⊆ 1..2 & y ⊆ 1..2 & x ∪ y = 1..2

TRUE

Solution:
	x = ∅
	y = {1,2}

Internally, the constraint is rewritten to support operators which do not exist in SMTLib:

In [64]:
∀ smt_tmp28.(smt_tmp28 ∈ x ⇒ smt_tmp28 ≥ 1 & 2 ≥ smt_tmp28) & 
∀ smt_tmp29.(smt_tmp29 ∈ y ⇒ smt_tmp29 ≥ 1 & 2 ≥ smt_tmp29) &
x ∪ y = {1,2}

TRUE

Solution:
	x = ∅
	y = {1,2}

This in turn gets translated to SMTLib (calls to the Z3 API):
- mk_var(set(integer),x) $\rightarrow$ 2
- mk_var(set(integer),y) $\rightarrow$ 3
- mk_bounded_var(integer,_smt_tmp28) $\rightarrow$ 4
- mk_op(member,4,2) $\rightarrow$ 5
- mk_int_const(1) $\rightarrow$ 6
- mk_op(greater_equal,4,6) $\rightarrow$ 7
- mk_int_const(2) $\rightarrow$ 8
- mk_op(greater_equal,8,4) $\rightarrow$ 9
- mk_op_arglist(conjunct,[7,9]) $\rightarrow$ 10
- mk_op(implication,5,10) $\rightarrow$ 11
- mk_quantifier(forall,[4],11) $\rightarrow$ 12
- mk_bounded_var(integer,_smt_tmp29) $\rightarrow$ 13
- mk_op(member,13,3) $\rightarrow$ 14
- mk_int_const(1) $\rightarrow$ 15
- mk_op(greater_equal) $\rightarrow$ 13,15,16
- mk_int_const(2) $\rightarrow$ 17
- mk_op(greater_equal,17,13) $\rightarrow$ 18
- mk_op_arglist(conjunct,[16,18]) $\rightarrow$ 19
- mk_op(implication,14,19) $\rightarrow$ 20
- mk_quantifier(forall,[13],20) $\rightarrow$ 21
- mk_op(union,2,3) $\rightarrow$ 22
- mk_int_const(1) $\rightarrow$ 23
- mk_int_const(2) $\rightarrow$ 24
- mk_set([23,24]) $\rightarrow$ 25
- mk_op(equal,22,25) $\rightarrow$ 26
- mk_op_arglist(conjunct,[12,21,26]) $\rightarrow$ 27

This can be solved by Z3 but not by CVC4. Already the slightly more complicated example from above (or the other examples) cannot be solved:

In [65]:
:solve z3 x ⊆ 1..2 & y ⊆ 1..2 & x ∪ y = 1..2 & 1∈x & x ⊂ y

[2018-06-04 14:02:49,960, T+539809] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] z3exception in query: canceled


CommandExecutionException: :solve: Computation not completed: time out

Let us look at another relatively simple example which poses problems:

In [66]:
:solve z3 f = {1|->3, 2|->6} & r = f~[{6}]

CommandExecutionException: :solve: Computation not completed: no solution found (but one might exist)

To understand why this simple constraint cannot be solved, we have to know how the translation works:
The relational inverse gets translated into two universal quantifications for SMTLib:
```
 x = y~
<=>
 !(st11,st12).(st11 |-> st12 : x => st12 |-> st11 : y) & 
 !(st11,st12).(st12 |-> st11 : y => st11 |-> st12 : x))
```
Similarly, r = f[s] is translated as follows:
```
 r = f[s]
<=>
 !st27.(st27 : r => #st26.(st26 |-> st27 : f & st26 : s) & 
 !st27.(#st26.(st26 |-> st27 : f & st26 : s) => st27 : r)
```
The resulting predicate (without the inverse and image operators) is the following, which Z3 cannot solve (but ProB can).

In [67]:
:prettyprint f = {(1|->3),(2|->6)} &
#st13.(r = st13 & (
    !st15.(st15 : st13 => #st14.(#st16.(st14 |-> st15 : st16 & 
    (!(st17,st18).(st17 |-> st18 : st16 => st18 |-> st17 : f) & 
     !(st17,st18).(st18 |-> st17 : f => st17 |-> st18 : st16))) & st14 : {6})) & 
     !st15.(#st14.(#st19.(st14 |-> st15 : st19 & (!(st20,st21).(st20 |-> st21 : st19 => st21 |-> st20 : f) &
     !(st20,st21).(st21 |-> st20 : f => st20 |-> st21 : st19))) & st14 : {6}) => st15 : st13)))

f = {(1↦3),(2↦6)} ∧ (∃ /* LET */ (st13).( (st13)=r ∧ ∀st15Â·(st15 ∈ st13 ⇒ ∃st16Â·(6 ↦ st15 ∈ st16 ∧ (∀(st17,st18)Â·(st17 ↦ st18 ∈ st16 ⇒ st18 ↦ st17 ∈ f) ∧ ∀(st17,st18)Â·(st18 ↦ st17 ∈ f ⇒ st17 ↦ st18 ∈ st16)))) ∧ ∀st15Â·(∃st19Â·(6 ↦ st15 ∈ st19 ∧ (∀(st20,st21)Â·(st20 ↦ st21 ∈ st19 ⇒ st21 ↦ st20 ∈ f) ∧ ∀(st20,st21)Â·(st21 ↦ st20 ∈ f ⇒ st20 ↦ st21 ∈ st19))) ⇒ st15 ∈ st13)))

In [68]:
:time :solve prob f = {(1|->3),(2|->6)} &
#st13.(r = st13 & (
    !st15.(st15 : st13 => #st14.(#st16.(st14 |-> st15 : st16 & 
    (!(st17,st18).(st17 |-> st18 : st16 => st18 |-> st17 : f) & 
     !(st17,st18).(st18 |-> st17 : f => st17 |-> st18 : st16))) & st14 : {6})) & 
     !st15.(#st14.(#st19.(st14 |-> st15 : st19 & (!(st20,st21).(st20 |-> st21 : st19 => st21 |-> st20 : f) &
     !(st20,st21).(st21 |-> st20 : f => st20 |-> st21 : st19))) & st14 : {6}) => st15 : st13)))


Execution time: 0.038201131 seconds


TRUE

Solution:
	r = {2}
	f = {(1↦3),(2↦6)}

In [69]:
:time :solve cvc4 f = {(1|->3),(2|->6)} &
#st13.(r = st13 & (
    !st15.(st15 : st13 => #st14.(#st16.(st14 |-> st15 : st16 & 
    (!(st17,st18).(st17 |-> st18 : st16 => st18 |-> st17 : f) & 
     !(st17,st18).(st18 |-> st17 : f => st17 |-> st18 : st16))) & st14 : {6})) & 
     !st15.(#st14.(#st19.(st14 |-> st15 : st19 & (!(st20,st21).(st20 |-> st21 : st19 => st21 |-> st20 : f) &
     !(st20,st21).(st21 |-> st20 : f => st20 |-> st21 : st19))) & st14 : {6}) => st15 : st13)))

CommandExecutionException: :time: :solve: Computation not completed: no solution found (but one might exist)

The SMTLib translation is still of limited value for finding models.
However, for finding inconsistencies it is much better and can detect certain inconsistencies which ProB's solver cannot.
While both ProB and Z3 can solve the following:

In [70]:
:solve prob x:s1 & x:s2 & x /: (s1 /\ s2) & s1 <: INTEGER

FALSE

only the Z3 backend can solve this one:

In [71]:
:solve z3 x:s1 & x/:s2 & x /: (s1 \/s2) & s1 <: INTEGER

FALSE

### ProB's Set Solver ###
ProB has actually three set representations:
- Prolog lists of elements
- AVL trees for fully known sets
- symbolic closures for large or infinite sets

For finite sets, the AVL tree representation is the most efficient and allows for efficient lookups.
It, however, requires all elements to be fully known.

The symbolic closure can be used for large or infinite sets.
ProB will automatically use it for sets it knows to be infinite, or when an enumeration warning occurs during an attempt at expanding a set.

The list representation is used for sets where some of the members are known or partially known.

#### AVL tree representation ####
The following generates the AVL tree representation:

In [72]:
{x|x∈0..2**10 & x mod 100 = 0}

{0,100,200,300,400,500,600,700,800,900,1000}

A lot of operators and predicates have optimised versions for the AVL tree represenation, e.g.,

In [73]:
s = {x|x∈0..2**10 & x mod 100 = 0} &
mx = max(s) &
mn = min(s)

TRUE

Solution:
	mn = 0
	s = {0,100,200,300,400,500,600,700,800,900,1000}
	mx = 1000

#### Symbolic closure representation ####

In the following case, ProB knows that the set is infinite and is kept symbolic:

In [74]:
{x|x>1000}

{x∣x > 1000}

Symbolic sets can be used in various ways:

In [75]:
inf = {x|x>1000} & 1024 : inf & not(1000:inf) & res  = (900..1100) ∩ inf

TRUE

Solution:
	inf = {x∣x > 1000}
	res = (1001 ‥ 1100)

For the following set, ProB tries to expand it and then an enumeration warning occurs (also called a **virtual timeout**, ProB realises that no matter what time budget it would be given a timeout would occur).
The set is then kept symbolic and can again be used in various ways.

In [76]:
inf = {x|x>1000 & x mod 25 = 0} & 1025 ∈ inf & not(1000∈inf) & res  = (900..1100) ∩ inf

[2018-06-04 14:03:14,350, T+564199] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] VIRTUAL TIME-OUT caused by: ### Warning: enumerating x : (all_solutions) : INTEGER : 1025:sup ---> 1025:1025


TRUE

Solution:
	inf = {x∣x > 1000 ∧ x mod 25 = 0}
	res = ((900 ‥ 1100) ∩ {x∣x > 1000 ∧ x mod 25 = 0})

The virtual timeout message can be removed (and performance improved) by adding the symbolic pragma:

In [77]:
inf = /*@symbolic*/ {x|x>1000 & x mod 25 = 0} & 1025 ∈ inf & not(1000∈inf) & res  = (900..1100) ∩ inf

TRUE

Solution:
	inf = {x∣x > 1000 ∧ x mod 25 = 0}
	res = ((900 ‥ 1100) ∩ {x∣x > 1000 ∧ x mod 25 = 0})

Internally, a symbolic representation is a **closure** in functional programming terms: all dependent variables are *compiled* into the closure: the closure can be passed as a value and evaluated without needing access to an environment. In Prolog this is represented as a tuple:
- closure(Parameters,Types,CompiledPredicate)
For example, a set {x|x>v} where v has the value 17 is compiled to:
- closure([x],[integer],```x>17```)

#### List representation ####
The list representation is used when a finite set is partially known and constraint solving has to determine the set.


In [78]:
vec: 1..10 --> 0..9999 &
vec(1) : {1,10} &
!x.(x:2..10 => vec(x) = vec(x-1)*2)

TRUE

Solution:
	vec = {(1↦1),(2↦2),(3↦4),(4↦8),(5↦16),(6↦32),(7↦64),(8↦128),(9↦256),(10↦512)}

Note that Kodkod translation and SMT translation not very effective for the above.
The Kodkod translation can deal with a simpler version of the above:

In [79]:
:time :solve kodkod vec: 1..8 --> 0..199 & vec(1) : {1,10} & !x.(x:2..8 => vec(x) = vec(x-1)*2)

[2018-06-04 14:03:20,699, T+570548] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   vec : 1 .. 8 --> 0 .. 199 & vec(1) : {1,10} & !x.(...  ints: irange(0,398), intatoms: irange(0,199)
[2018-06-04 14:03:20,699, T+570548] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Times for computing solutions: [204]
Execution time: 2.184422799 seconds


TRUE

Solution:
	vec = {(3↦4),(5↦16),(6↦32),(7↦64),(1↦1),(2↦2),(4↦8),(8↦128)}

In [80]:
:time :solve prob vec: 1..8 --> 0..199 & vec(1) : {1,10} & !x.(x:2..8 => vec(x) = vec(x-1)*2)

Execution time: 0.014949467 seconds


TRUE

Solution:
	vec = {(1↦1),(2↦2),(3↦4),(4↦8),(5↦16),(6↦32),(7↦64),(8↦128)}

In [81]:
:time :solve z3 vec: 1..8 --> 0..199 & vec(1) : {1,10} & !x.(x:2..8 => vec(x) = vec(x-1)*2)

[2018-06-04 14:03:24,797, T+574646] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] z3exception in query: canceled


CommandExecutionException: :time: :solve: Computation not completed: time out

### ProB's Solving Algorithm ###

ProB tries to accomplish several conflicting goals:
- being able to deal with concrete data, i.e., sets and relations containing thousands or hundreds of thousands of elementas
- being able to deal with symbolic, infinite sets, relations and functions.
- being able to perform efficient computation over large data as well as constraint solving

For example, efficient computation over large concrete data is the following:

In [89]:
:time :solve prob s1 = {x|x:1..10**n & x mod n = 0} & s2 = {y|y:1..10**n & y mod (n+1) = 0} & s3 = s1 /\ s2 & n=4

Execution time: 0.305008223 seconds


TRUE

Solution:
	s3 = ∃500∈{20,40,…,9980,10000}
	n = 4
	s1 = ∃2500∈{4,8,…,9996,10000}
	s2 = ∃2000∈{5,10,…,9995,10000}

Here is a simple verison of the above

In [90]:
:time :solve prob x = 1..n & y = 2*n..3*n & n = 100 & xy = x \/ y

Execution time: 0.013053703 seconds


TRUE

Solution:
	xy = ∃201∈{1,2,…,299,300}
	x = (1 ‥ 100)
	y = (200 ‥ 300)
	n = 100

In [91]:
:solve z3 x = 1..n & y = 2*n..3*n & n = 100 & xy = x \/ y

CommandExecutionException: :solve: Computation not completed: time out

In [92]:
:time :solve kodkod x = 1..n & y = 2*n..3*n & n = 100 & xy = x \/ y

[2018-06-04 14:05:05,176, T+675025] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   x = 1 .. n & y = 2 * n .. 3 * n & n = 100 & xy = x...  ints: irange(1,300), intatoms: irange(1,300)
[2018-06-04 14:05:05,176, T+675025] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Times for computing solutions: [2]
Execution time: 0.336676466 seconds


TRUE

Solution:
	xy = {3,5,6,7,9,10,11,12,13,14,15,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,200,201,202,203,204,205,206,…}
	x = {3,5,6,7,9,10,11,12,13,14,15,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,1,2,4,8,16,32,64}
	y = {200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,29

In the appendix there are more examples which analyse the performance for such examples.

ProB employs the *Andorra* principle: deterministic computations are done first.
As there are multiple set representations, there are actually two kinds of deterministic computations:
- deterministic computations that generate an efficient representation, e.g., an AVL set representation
- and other deterministic computations

The ProB solver has a **WAITFLAG store** where choice points and enumerations are registered with a given priority.
- Priority 0 means that an efficient representation can be generated
- Priority 1 is a deterministic computation not guaranteed to produce an efficient representation
- Priority k is a choice point/enumeration which may generate k possible values

At each solving step one waitflag is activated, the one with the lowest priority.
CLP(FD) variables are also registered in the WAITFLAG store and are enumerated before a waitflag of the same priority is activated. For tie breaking one typically uses the **most attached constraints first** (ffc) heuristic.

#### Example ####
Let us examine how ```x = 1..n & y = 2*n..3*n & n = 100 & xy = x \/ y``` is solved.
- all constraints are registered
- in phase 0 ```n=100``` is run
- this means that ```1..n``` can be efficiently computed
- this means that ```x = 1..n``` triggers in phase 0
- then ```2*n``` and ```3*n``` can be computed, followed by ```2*n..3*n```
- this means that ```y = 2*n..3*n``` triggers in phase 0
- again, this means that ```x \/ y``` can be efficiently computed
- finally ```xy = x \/ y``` can be executed in phase 0

No enumeration was required. In this case ProB's constraint solver works similar to a topological sorting algorithm.


#### Dealing with unbounded enumeration ####

Note: if an unbounded enumeration is encountered, the solver registers an **enumeration warning** in the current scope (every quantification / comprehension set results in a new inner scope). Depending on the kind of scope (existential/universal) and on whether a solution is found, the warning gets translated into an **UNKNOWN** result.

### Functional Programming ###

Some functions are automatically detected as infinite by ProB, are kept symbolic but can be applied in several ways:


In [93]:
f = %x.(x:INTEGER|x*x) &
r1 = f(100000) &
r2 = f[1..10] &
r3 = ([2,3,5,7,11] ; f) &
r4 = iterate(f,3)(2) &
f(sqrt) = 100

TRUE

Solution:
	r2 = {1,4,9,16,25,36,49,64,81,100}
	r3 = [4,9,25,49,121]
	r4 = 256
	sqrt = 10
	f = λx·(x ∈ INTEGER∣x ∗ x)
	r1 = 10000000000

In [94]:
f = {x,y|x:NATURAL & y**2 >= x & (y-1)**2 <x } & // integer square root function
r1 = f(100000) &
r2 = f[1..10] &
r3 = ([2,3,5,7,11] ; f) &
r4 = iterate(f,3)(2) &
f(sqr) = 100 &
r5 = closure1(f)[{10000}]

[2018-06-04 14:08:39,826, T+889675] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] ! source(b_compute_comprehension_set)
[2018-06-04 14:08:39,826, T+889675] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] ! Keeping comprehension-set symbolic (you may want to use the /*@symbolic*/ pragma to prevent this message, unless it was due to a WD-Error), identifiers: [x,y]
[2018-06-04 14:08:39,827, T+889676] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] ! Line: 1 Column: 9 until 44
[2018-06-04 14:08:39,837, T+889686] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] VIRTUAL TIME-OUT caused by: ### Warning: enumerating NATURAL(1) : NATURAL(1) : 1:sup ---> 1:3


TRUE

Solution:
	r2 = {1,2,3,4}
	r3 = [2,2,3,3,4]
	r4 = 2
	r5 = {2,4,10,100}
	sqr = 9802
	f = {x,y∣x ∈ NATURAL ∧ y × 2 ≥ x ∧ (y − 1) × 2 < x}
	r1 = 317

### Reification ###

Reification is linking the truth value of a constraint with a boolean variable.
ProB's kernel provides support for reifying a considerable number of constraints (but not yet all!).
Reification is important for efficiency, to avoid choice points and is important to link various solvers of ProB (set, arithmetic, boolean,...).
![CBCKernel](./img/ProB_CBC_Kernel.pdf)

In [95]:
(x>100 <=> (ReifVar=TRUE)) & (x<125 <=> (ReifVar=FALSE)) & x<200

TRUE

Solution:
	x = 125
	ReifVar = TRUE

![Linking](./img/Linking.png)

### Relation to SETLOG ###

Setlog (http://people.dmi.unipr.it/gianfranco.rossi/setlog.Home.html) is based on non-deterministic set unification
Setlog has additional inference rules

The former causes problems with larger sets, in our experience.
The latter could be added to ProB via CHR, but currently not done.

Let us look at a simpler Setlog example from the article "{log} as a Test Case Generator
for the Test Template Framework" by Cristia, Rossi and Frydman:
```
1 in R & 1 nin S & inters(R,S,T) & T = {X}
```
This can be encoded in B as follows:

In [96]:
1:R & 1/:S & R/\S=T & T={X}

TRUE

Solution:
	R = {1,0}
	S = {0}
	T = {0}
	X = 0

Another example from that paper is
```
X in int(1,5) & Y in int(4,10) & inters({X},{Y},R) & X >= Y
```
which has three solutions
```
X=4,Y=4,R={4}; X=5,Y=5,R={5}; X=5,Y=4,R={}.
```
Let us check this with ProB:

In [97]:
{X,Y,R|X: 1..5 & Y: 4..10 & {X}/\{Y}=R & X>=Y}

{((4↦4)↦{4}),((5↦4)↦∅),((5↦5)↦{5})}

However, in particular with unbounded sets Setlog can solve some constraints that ProB, Z3 and Kodkod cannot:
```
un(A,B,D) & disj(A,C) & D=C & ris(X in A,[],true,X) neq {}
```
In B this corresponds to:

In [98]:
A \/ B = D & A /\ C = {} & D=C & A /= {} & A:POW(STRING)

[2018-06-04 14:08:47,442, T+897291] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] ### Warning: enumerating A : STRING : inf ---> "STRING1","STRING2",...
[2018-06-04 14:08:47,443, T+897292] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] ### Warning: enumerating B : STRING : inf ---> "STRING1","STRING2",...


CommandExecutionException: :eval: UNKNOWN (FALSE with enumeration warning)

From A\/B=D we and D=C we could infer A<:C and hence A/\C=C and hence A={} which is in conflict with A/={}.
ProB (as well as Kodkod and Z3 backends) can only infer the conflict for finite domains:

In [99]:
A \/ B = D & A /\ C = {} & D=C & A /= {} & A:POW(BOOL)

FALSE

In [100]:
:solve kodkod A \/ B = D & A /\ C = {} & D=C & A /= {} & A:POW(BOOL)

[2018-06-04 14:08:48,854, T+898703] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   A \/ B = D & A /\ C = {} & D = C & A /= {}  ints: none, intatoms: none
[2018-06-04 14:08:48,854, T+898703] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Times for computing solutions: []


FALSE

In [101]:
:solve z3 A \/ B = D & A /\ C = {} & D=C & A /= {} & A:POW(BOOL)

FALSE

Setlog has problems with larger sets. For example, the following takes 24 seconds using the latest stable release 4.9.1 of Setlog from http://people.dmi.unipr.it/gianfranco.rossi/setlog.Home.html: 
```
{log}=> diff(int(1,200),{50},R).

R = {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200}

Another solution?  (y/n)
```
In ProB this is instantenous:

In [102]:
:time R= (1..200) \ {50}

Execution time: 0.005074849 seconds


TRUE

Solution:
	R = {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200}

## Summary of Set Constraint Solving Approaches ##

- SAT Translation (Kodkod backend): 
 - needs finite and small base type, no unbounded or higher-order sets
 - can be very effective for complex constraints involving image, transitive closure,...
 - limited performance for large sets
- SMTLib Translation (Z3/CVC4 backend):
 - can deal with unbounded and large sets
 - due to heavy use of quantifiers, some finite constraints get translated into infinite ones: limited model finding capabilities
- ProB's default backend:
 - can deal with unbounded and large sets
 - limited constraint solving for complex constraints involving image, transitive closure,...
 - no learning, backjumping
 - works well for large sets and semi-deterministic computation
 - works well for animation, data validation, disproving
 - limitations appear for symbolic model checking (IC3,...)
  - Future work: improve combination with Z3/Kodkod, improve list representation (maybe use a bit-vector like representation, and CLP(FD) cardinality variable) 
 

### Integrations of Approaches ###

ProB provides the joint application of the CLP(FD) and SMT backend (preference ```SMT_SUPPORTED_INTERPRETER```.
Constraints are posted both to ProB and Z3/CVC4, with the hope that Z3/CVC4 prune infeasible branches.
The main motivation was new symbolic validation techniques such as IC3.

The Kodkod integration also passes higher-order/unbounded constraints to ProB, after solving the first order finite constraints with Kodkod/Alloy.
However, this kind of integration is rarely useful (most of the generated solutions get rejected by ProB).

A more promising, fine-grained integration has been presented at PADL'18.

## Appendix ##

### Explicit Computations ####

What about explicit computations? How well does the SMTLib translation fare here?

In [103]:
:solve z3 x = 1..1000 /\ (200..300)

CommandExecutionException: :solve: Computation not completed: no solution found (but one might exist)

In [104]:
:time :solve z3 x = 1..40 /\ (6..15)

Execution time: 0.175097068 seconds


TRUE

Solution:
	x = {6,7,8,9,10,11,12,13,14,15}

In [105]:
:time :solve z3 x = 1..60 /\ (6..15)

Execution time: 0.754973438 seconds


TRUE

Solution:
	x = {6,7,8,9,10,11,12,13,14,15}

In [106]:
:time :solve z3 x = 1..80 /\ (6..15)

Execution time: 2.369086566 seconds


TRUE

Solution:
	x = {6,7,8,9,10,11,12,13,14,15}

In [107]:
:time :solve prob x = 1..80 /\ (6..15)

Execution time: 0.004883079 seconds


TRUE

Solution:
	x = (6 ‥ 15)

In the following the inverse operator seems to pose problems to Z3:

In [108]:
:solve z3 s1 = {2,3,5,7,11} & s2 = {4,8,16,32} & c = s1*s2 & r=c~[{8}]

CommandExecutionException: :solve: Computation not completed: no solution found (but one might exist)

In [109]:
:solve prob s1 = {2,3,5,7,11} & s2 = {4,8,16,32} & c = s1*s2 & r=c~[{8}]

TRUE

Solution:
	r = {2,3,5,7,11}
	c = ({2,3,5,7,11} ∗ {4,8,16,32})
	s1 = {2,3,5,7,11}
	s2 = {4,8,16,32}

### Comparison ###
Some further examples for comparison of the backends

In [110]:
:time :solve prob f: 1..n --> 1..n & !x.(x:2..n => f(x)=f(x-1)+1) & n=50

Execution time: 0.012805716 seconds


TRUE

Solution:
	f = {(1↦1),(2↦2),(3↦3),(4↦4),(5↦5),(6↦6),(7↦7),(8↦8),(9↦9),(10↦10),(11↦11),(12↦12),(13↦13),(14↦14),(15↦15),(16↦16),(17↦17),(18↦18),(19↦19),(20↦20),(21↦21),(22↦22),(23↦23),(24↦24),(25↦25),(26↦26),(27↦27),(28↦28),(29↦29),(30↦30),(31↦31),(32↦32),(33↦33),(34↦34),(35↦35),(36↦36),(37↦37),(38↦38),(39↦39),(40↦40),(41↦41),(42↦42),(43↦43),(44↦44),(45↦45),(46↦46),(47↦47),(48↦48),(49↦49),(50↦50)}
	n = 50

In [111]:
:time :solve kodkod f: 1..n --> 1..n & !x.(x:2..n => f(x)=f(x-1)+1) & n=50

[2018-06-04 14:09:10,559, T+920408] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   f : 1 .. n --> 1 .. n & !x.(x : 2 .. n => f(x) = f...  ints: irange(1,50), intatoms: irange(1,50)
[2018-06-04 14:09:10,560, T+920409] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Times for computing solutions: [1185]
Execution time: 3.657044854 seconds


TRUE

Solution:
	f = {(3↦3),(5↦5),(6↦6),(7↦7),(9↦9),(10↦10),(11↦11),(12↦12),(13↦13),(14↦14),(15↦15),(17↦17),(18↦18),(19↦19),(20↦20),(21↦21),(22↦22),(23↦23),(24↦24),(25↦25),(26↦26),(27↦27),(28↦28),(29↦29),(30↦30),(31↦31),(33↦33),(34↦34),(35↦35),(36↦36),(37↦37),(38↦38),(39↦39),(40↦40),(41↦41),(42↦42),(43↦43),(44↦44),(45↦45),(46↦46),(47↦47),(48↦48),(49↦49),(50↦50),(1↦1),(2↦2),(4↦4),(8↦8),(16↦16),(32↦32)}
	n = 50

In [112]:
:time :solve z3 f: 1..n --> 1..n & !x.(x:2..n => f(x)=f(x-1)+1) & n=50

[2018-06-04 14:09:13,146, T+922995] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] z3exception in query: canceled


CommandExecutionException: :time: :solve: Computation not completed: time out

### Fast with Kodkod ###

#### Example with relational composition and image ####

In [113]:
:time :solve prob r: 1..5 <-> 1..5 & (r;r) = r & r /= {} & dom(r)=1..5 & r[2..3]=3..4

Execution time: 2.087463022 seconds


TRUE

Solution:
	r = {(1↦1),(1↦2),(1↦3),(1↦4),(1↦5),(2↦3),(2↦4),(3↦3),(3↦4),(4↦3),(4↦4),(5↦1),(5↦2),(5↦3),(5↦4),(5↦5)}

In [114]:
:time :solve kodkod r: 1..5 <-> 1..5 & (r;r) = r & r /= {} & dom(r)=1..5 & r[2..3]=3..4

[2018-06-04 14:09:15,382, T+925231] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   r : 1 .. 5 <-> 1 .. 5 & (r ; r) = r & r /= {} & do...  ints: irange(0,5), intatoms: irange(0,5)
[2018-06-04 14:09:15,383, T+925232] "ProB Output Logger for instance 1ede3ed1" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Times for computing solutions: [1,1,1,0,1,0,1,0,1,1,0,1,0,1,1,0,1,0,1,0,1,1]
Execution time: 0.037624591 seconds


TRUE

Solution:
	r = {(3↦3),(3↦4),(5↦4),(1↦4),(2↦4),(4↦4)}

In [115]:
:time :solve z3 r: 1..5 <-> 1..5 & (r;r) = r & r /= {} & dom(r)=1..5 & r[2..3]=3..4

CommandExecutionException: :time: :solve: Computation not completed: no solution found (but one might exist)

In [116]:
:time :solve prob r: 1..5 <-> 1..5 & (r;r) = r & r /= {} & dom(r)=1..5 & r[2..3]=3..4

Execution time: 1.971222669 seconds


TRUE

Solution:
	r = {(1↦1),(1↦2),(1↦3),(1↦4),(1↦5),(2↦3),(2↦4),(3↦3),(3↦4),(4↦3),(4↦4),(5↦1),(5↦2),(5↦3),(5↦4),(5↦5)}

#### Graph theorem ####

Theorem: all undirected graphs without self-loops (and no 0-vertices) have two nodes with the same degree.

In [10]:
::load
MACHINE GraphTheorem
SETS NODES5 
PROPERTIES card(NODES5)=5
END

[2018-06-04 16:46:59,166, T+6475117] "Shell-0" de.prob.cli.PrologProcessProvider.makeProcess(PrologProcessProvider.java:64): [INFO] Starting ProB's Prolog Core. Path is /Users/leuschel/git_root/prob_prolog/probcli.sh
[2018-06-04 16:47:00,347, T+6476298] "Shell-0" de.prob.cli.PortPattern.setValue(PortPattern.java:30): [INFO] Server has started and listens on port 55595
[2018-06-04 16:47:00,348, T+6476299] "Shell-0" de.prob.cli.InterruptRefPattern.setValue(InterruptRefPattern.java:29): [INFO] Server can receive user interrupts via reference 52301
[2018-06-04 16:47:00,350, T+6476301] "ProB Output Logger for instance ecdd0f" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] -- starting command loop --
[2018-06-04 16:47:00,364, T+6476315] "ProB Output Logger for instance ecdd0f" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Connected: 127.0.0.1


Loaded machine: GraphTheorem : []


In [118]:
:prettyprint edges : NODES5 <-> NODES5 & 
      edges~=edges &
      not(#(n1,n2).(n1:NODES5 & n2:NODES5 & n2/=n1 & 
      card(edges[{n1}]) = card(edges[{n2}]))) &
      dom(edges)=NODES5 & id(NODES5) /\ edges = {}

edges ∈ NODES5 ↔ NODES5 ∧ edges⁻¹ = edges ∧ ¬(∃(n1,n2)Â·(n1 ∈ NODES5 ∧ n2 ∈ NODES5 ∧ n2 ≠ n1 ∧ card(edges[{n1}]) = card(edges[{n2}]))) ∧ dom(edges) = NODES5 ∧ id(NODES5) ∩ edges = ∅

In [119]:
:time :solve prob  edges : NODES5 <-> NODES5 & 
      edges~=edges &
      not(#(n1,n2).(n1:NODES5 & n2:NODES5 & n2/=n1 & 
      card(edges[{n1}]) = card(edges[{n2}]))) &
      dom(edges)=NODES5 & id(NODES5) /\ edges = {}

CommandExecutionException: :time: :solve: Computation not completed: time out

In [120]:
:time :solve kodkod  edges : NODES5 <-> NODES5 & 
      edges~=edges &
      not(#(n1,n2).(n1:NODES5 & n2:NODES5 & n2/=n1 & 
      card(edges[{n1}]) = card(edges[{n2}]))) &
      dom(edges)=NODES5 & id(NODES5) /\ edges = {}

[2018-06-04 14:09:25,781, T+935630] "ProB Output Logger for instance 2d81a536" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   edges~ = edges & not(#(n1,n2).(n2 /= n1 & card(edg...  ints: irange(0,5), intatoms: none
[2018-06-04 14:09:25,781, T+935630] "ProB Output Logger for instance 2d81a536" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Kodkod module started up successfully (SAT solver SAT4J with timeout of 1500 ms).
[2018-06-04 14:09:25,782, T+935631] "ProB Output Logger for instance 2d81a536" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Times for computing solutions: []
Execution time: 0.837816834 seconds


FALSE

In [121]:
:time :solve z3  edges : NODES5 <-> NODES5 & 
      edges~=edges &
      not(#(n1,n2).(n1:NODES5 & n2:NODES5 & n2/=n1 & 
      card(edges[{n1}]) = card(edges[{n2}]))) &
      dom(edges)=NODES5 & id(NODES5) /\ edges = {}

CommandExecutionException: :time: :solve: Computation not completed: no solution found (but one might exist)

So with Kodkod we proved the theorem for 5 nodes in about a second.
What if we remove the self-loops restriction:

In [122]:
:time :solve kodkod  edges : NODES5 <-> NODES5 & 
      edges~=edges &
      not(#(n1,n2).(n1:NODES5 & n2:NODES5 & n2/=n1 & 
      card(edges[{n1}]) = card(edges[{n2}]))) &
      dom(edges)=NODES5

[2018-06-04 14:09:26,058, T+935907] "ProB Output Logger for instance 2d81a536" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   edges~ = edges & not(#(n1,n2).(n2 /= n1 & card(edg...  ints: irange(0,5), intatoms: none
[2018-06-04 14:09:26,059, T+935908] "ProB Output Logger for instance 2d81a536" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Times for computing solutions: [5,4,3,1,2,2,2,2,2,1,7,1,2,2,2,1,3,2,2,2,1,2]
Execution time: 0.107056638 seconds


TRUE

Solution:
	edges = {(NODES51↦NODES54),(NODES52↦NODES52),(NODES52↦NODES53),(NODES52↦NODES54),(NODES53↦NODES52),(NODES53↦NODES53),(NODES53↦NODES54),(NODES53↦NODES55),(NODES54↦NODES51),(NODES54↦NODES52),(NODES54↦NODES53),(NODES54↦NODES54),(NODES54↦NODES55),(NODES55↦NODES53),(NODES55↦NODES54)}

In [123]:
:time :solve kodkod  edges : NODES5 <-> NODES5 & 
      edges~=edges &
      not(#(n1,n2).(n1:dom(edges) & n2:dom(edges) & n2/=n1 & 
                    card(edges[{n1}]) = card(edges[{n2}]))) &
      id(NODES5) /\ edges = {} & edges /= {}

[2018-06-04 14:09:26,173, T+936022] "ProB Output Logger for instance 2d81a536" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   edges~ = edges & not(#(n1,n2).(n1 : dom(edges) & n...  ints: irange(0,5), intatoms: none
Execution time: 0.056822248 seconds


FALSE

### Benchmark Puzzles ####

#### N-Queens ####


In [124]:
:time :solve prob n=8 &
    queens : 1..n >-> 1..n &
    !(q1,q2).(q1:1..n & q2:2..n & q2>q1 => queens(q1)+(q2-q1) /= queens(q2) & queens(q1)+(q1-q2) /= queens(q2))

Execution time: 0.053214873 seconds


TRUE

Solution:
	queens = {(1↦4),(2↦2),(3↦7),(4↦3),(5↦6),(6↦8),(7↦5),(8↦1)}
	n = 8

In [125]:
:time :solve z3 n=8 &
    queens : 1..n >-> 1..n &
    !(q1,q2).(q1:1..n & q2:2..n & q2>q1 => queens(q1)+(q2-q1) /= queens(q2) & queens(q1)+(q1-q2) /= queens(q2))

Execution time: 0.370806337 seconds


TRUE

Solution:
	queens = {(1↦4),(2↦7),(3↦5),(4↦2),(5↦6),(6↦1),(7↦3),(8↦8)}
	n = 8

In [126]:
:time :solve kodkod n=8 &
    queens : 1..n >-> 1..n &
    !(q1,q2).(q1:1..n & q2:2..n & q2>q1 => queens(q1)+(q2-q1) /= queens(q2) & queens(q1)+(q1-q2) /= queens(q2))

[2018-06-04 14:09:27,167, T+937016] "ProB Output Logger for instance 2d81a536" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   n = 8 & queens : 1 .. n >-> 1 .. n & !(q1,q2).(q1 ...  ints: irange(-7,15), intatoms: irange(1,8)
[2018-06-04 14:09:27,168, T+937017] "ProB Output Logger for instance 2d81a536" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Times for computing solutions: [27,19,44,9,6,6,11,5,5,10,4,9,22,2,6,8,4,3,3,14,3,3]
Execution time: 0.405027874 seconds


TRUE

Solution:
	queens = {(3↦2),(5↦5),(6↦1),(7↦8),(1↦3),(2↦6),(4↦7),(8↦4)}
	n = 8

#### N-Bishops ####


In [127]:
:time :solve prob n=3 & bshp <: (1..n)*(1..n) & 
  !(i,j).({i,j}<:1..n => ( (i,j): bshp => (!k.(k:(i+1)..n => (k,j+k-i) /: bshp & (k,j-k+i) /: bshp )) )) &
  card(bshp) = 3

Execution time: 0.054922769 seconds


TRUE

Solution:
	bshp = {(1↦1),(1↦2),(1↦3)}
	n = 3

In [128]:
:time :solve z3 n=3 & bshp <: (1..n)*(1..n) & 
  !(i,j).({i,j}<:1..n => ( (i,j): bshp => (!k.(k:(i+1)..n => (k,j+k-i) /: bshp & (k,j-k+i) /: bshp )) )) &
  card(bshp) = 3

[2018-06-04 14:09:29,856, T+939705] "ProB Output Logger for instance 2d81a536" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] z3exception in query: canceled


CommandExecutionException: :time: :solve: Computation not completed: time out

Solving takes about 150 seconds; Kodkod translation currently does not work due to card and preventing overflows.

In [129]:
:prettyprint n=3 & bshp <: (1..n)*(1..n)  & 
  !(i,j).({i,j}<:1..n => ( (i,j): bshp => (!k.(k:(i+1)..n => (k,j+k-i) /: bshp & (k,j-k+i) /: bshp )) )) &
  card(bshp) = 3

n = 3 ∧ bshp ⊆ (1 ‥ n) × (1 ‥ n) ∧ ∀(i,j)Â·({i,j} ⊆ 1 ‥ n ⇒ (i ↦ j ∈ bshp ⇒ ∀kÂ·(k ∈ i + 1 ‥ n ⇒ (k ↦ (j + k) - i) ∉ bshp ∧ (k ↦ (j - k) + i) ∉ bshp))) ∧ card(bshp) = 3

### Other examples ###


In [130]:
!x.(x:1..2 => !y.(y:1..3 => (x,y):r)) 

TRUE

Solution:
	r = {(1↦1),(1↦2),(1↦3),(2↦1),(2↦2),(2↦3)}

In [131]:
:solve z3 !x.(x:1..2 => !y.(y:1..3 => (x,y):r)) 

TRUE

Solution:
	r = {z_∣z_ ∈ INTEGER ∗ INTEGER}

For unbounded problem above, Z3 can find a model, but for the one below not (even with mbqi and pull_nested_quantifiers).

In [132]:
:solve z3 !x.(x:1..2 => !y.(y:1..3 => (x,y):r)) & r<: (1..10)*(1..10)

CommandExecutionException: :solve: Computation not completed: no solution found (but one might exist)

In [133]:
:time :solve kodkod !x.(x:1..2 => !y.(y:1..3 => (x,y):r)) & r<: (1..10)*(1..10)

[2018-06-04 14:09:32,837, T+942686] "ProB Output Logger for instance 2d81a536" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   !x.(x : 1 .. 2 => !y.(y : 1 .. 3 => x |-> y : r)) ...  ints: irange(1,10), intatoms: irange(1,10)
[2018-06-04 14:09:32,839, T+942688] "ProB Output Logger for instance 2d81a536" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Times for computing solutions: [2,2,0,2,3,6,0,4,2,0,1,1,2,1,2,0,1,2,4,2,0,1]
Execution time: 0.112904757 seconds


TRUE

Solution:
	r = {(1↦3),(1↦1),(1↦2),(2↦3),(2↦1),(2↦2)}

In [134]:
:time :solve prob !x.(x:1..2 => !y.(y:1..3 => (x,y):r)) & r<: (1..10)*(1..10)

Execution time: 0.019540215 seconds


TRUE

Solution:
	r = {(1↦1),(1↦2),(1↦3),(2↦1),(2↦2),(2↦3)}

#### Datavalidation example ####


In [135]:
::load
MACHINE
    signals

SETS
/* Ensemble des signaux */
   	SIGNAL =
   	{    PL01
   	,    PL02
   	,    PL03
   	,    PL04
   	,    PL05
   	,    PL06
   	,    PL07
   	,    PL08
   	,    PL09
   	,    PL10
   	,    PL11
   	,    PL12
	,	 PL13
   	,    PL14
   	,    PL15
   	,    PL16
   	,    PL17
   	,    PL18
   	,    PL19
   	,    PL20
   	}
END

[2018-06-04 14:09:33,012, T+942861] "Shell-0" de.prob.cli.PrologProcessProvider.makeProcess(PrologProcessProvider.java:64): [INFO] Starting ProB's Prolog Core. Path is /Users/leuschel/git_root/prob_prolog/probcli.sh
[2018-06-04 14:09:34,585, T+944434] "Shell-0" de.prob.cli.PortPattern.setValue(PortPattern.java:30): [INFO] Server has started and listens on port 63565
[2018-06-04 14:09:34,585, T+944434] "Shell-0" de.prob.cli.InterruptRefPattern.setValue(InterruptRefPattern.java:29): [INFO] Server can receive user interrupts via reference 47984
[2018-06-04 14:09:34,589, T+944438] "ProB Output Logger for instance 79d26131" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] -- starting command loop --
[2018-06-04 14:09:34,613, T+944462] "ProB Output Logger for instance 79d26131" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Connected: 127.0.0.1


Loaded machine: signals : []


In [136]:
:time :solve prob nxt = 
    {PL01 |-> PL02, PL02 |-> PL03, PL03 |-> PL04, PL04 |-> PL05,
     PL05 |-> PL06, PL06 |-> PL07, PL07 |-> PL08, PL08 |-> PL09,
     PL09 |-> PL10, PL10 |-> PL11, PL11 |-> PL11, PL12 |-> PL13,
     PL13 |-> PL14, PL14 |-> PL15, PL15 |-> PL16, PL16 |-> PL17,
     PL17 |-> PL18, PL18 |-> PL19, PL19 |-> PL20, PL20 |-> PL20} &
     res = SIGNAL \ nxt[SIGNAL]

Execution time: 0.201494972 seconds


TRUE

Solution:
	res = {PL01,PL12}
	nxt = {(PL01↦PL02),(PL02↦PL03),(PL03↦PL04),(PL04↦PL05),(PL05↦PL06),(PL06↦PL07),(PL07↦PL08),(PL08↦PL09),(PL09↦PL10),(PL10↦PL11),(PL11↦PL11),(PL12↦PL13),(PL13↦PL14),(PL14↦PL15),(PL15↦PL16),(PL16↦PL17),(PL17↦PL18),(PL18↦PL19),(PL19↦PL20),(PL20↦PL20)}

In [137]:
:time :solve z3 nxt = 
    {PL01 |-> PL02, PL02 |-> PL03, PL03 |-> PL04, PL04 |-> PL05,
     PL05 |-> PL06, PL06 |-> PL07, PL07 |-> PL08, PL08 |-> PL09,
     PL09 |-> PL10, PL10 |-> PL11, PL11 |-> PL11, PL12 |-> PL13,
     PL13 |-> PL14, PL14 |-> PL15, PL15 |-> PL16, PL16 |-> PL17,
     PL17 |-> PL18, PL18 |-> PL19, PL19 |-> PL20, PL20 |-> PL20} &
     res = SIGNAL \ nxt[SIGNAL]

CommandExecutionException: :time: :solve: Computation not completed: no solution found (but one might exist)

In [138]:
:time :solve kodkod nxt = 
    {PL01 |-> PL02, PL02 |-> PL03, PL03 |-> PL04, PL04 |-> PL05,
     PL05 |-> PL06, PL06 |-> PL07, PL07 |-> PL08, PL08 |-> PL09,
     PL09 |-> PL10, PL10 |-> PL11, PL11 |-> PL11, PL12 |-> PL13,
     PL13 |-> PL14, PL14 |-> PL15, PL15 |-> PL16, PL16 |-> PL17,
     PL17 |-> PL18, PL18 |-> PL19, PL19 |-> PL20, PL20 |-> PL20} &
     res = SIGNAL \ nxt[SIGNAL]

[2018-06-04 14:09:38,479, T+948328] "ProB Output Logger for instance 79d26131" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod ok:   nxt = {PL01 |-> PL02,PL02 |-> PL03,PL03 |-> PL04,P...  ints: none, intatoms: none
[2018-06-04 14:09:38,480, T+948329] "ProB Output Logger for instance 79d26131" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Kodkod module started up successfully (SAT solver SAT4J with timeout of 1500 ms).
[2018-06-04 14:09:38,480, T+948329] "ProB Output Logger for instance 79d26131" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Times for computing solutions: [2]
Execution time: 0.686312764 seconds


TRUE

Solution:
	res = {PL01,PL12}
	nxt = {(PL01↦PL02),(PL02↦PL03),(PL03↦PL04),(PL04↦PL05),(PL05↦PL06),(PL06↦PL07),(PL07↦PL08),(PL08↦PL09),(PL09↦PL10),(PL10↦PL11),(PL11↦PL11),(PL12↦PL13),(PL13↦PL14),(PL14↦PL15),(PL15↦PL16),(PL16↦PL17),(PL17↦PL18),(PL18↦PL19),(PL19↦PL20),(PL20↦PL20)}

In [4]:
::load
MACHINE
    signals

SETS
/* Ensemble des signaux */
   	SIGNAL =
   	{    PL01
   	,    PL02
   	,    PL03
   	,    PL04
   	,    PL05
   	,    PL06
   	,    PL07
   	,    PL08
   	,    PL09
   	,    PL10
   	,    PL11
   	,    PL12
	,	 PL13
   	,    PL14
   	,    PL15
   	,    PL16
   	,    PL17
   	,    PL18
   	,    PL19
   	,    PL20
   	,    PL21
   	,    PL22
	,	 PL23
   	,    PL24
   	,    PL25
   	,    PL26
   	,    PL27
   	,    PL28
   	,    PL29
   	,    PL30
   	,    PL31
   	,    PL32
	,	 PL33
   	,    PL34
   	,    PL35
   	,    PL36
   	,    PL37
   	,    PL38
   	,    PL39
   	,    PL40
   	}
END

[2018-06-04 15:03:05,856, T+241807] "Shell-0" de.prob.cli.PrologProcessProvider.makeProcess(PrologProcessProvider.java:64): [INFO] Starting ProB's Prolog Core. Path is /Users/leuschel/git_root/prob_prolog/probcli.sh
[2018-06-04 15:03:07,015, T+242966] "Shell-0" de.prob.cli.PortPattern.setValue(PortPattern.java:30): [INFO] Server has started and listens on port 51091
[2018-06-04 15:03:07,018, T+242969] "Shell-0" de.prob.cli.InterruptRefPattern.setValue(InterruptRefPattern.java:29): [INFO] Server can receive user interrupts via reference 50312
[2018-06-04 15:03:07,021, T+242972] "ProB Output Logger for instance 5f836b9a" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] -- starting command loop --
[2018-06-04 15:03:07,034, T+242985] "ProB Output Logger for instance 5f836b9a" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Connected: 127.0.0.1


Loaded machine: signals : []


In [140]:
:time :solve kodkod nxt = 
    {PL01 |-> PL02, PL02 |-> PL03, PL03 |-> PL04, PL04 |-> PL05,
     PL05 |-> PL06, PL06 |-> PL07, PL07 |-> PL08, PL08 |-> PL09,
     PL09 |-> PL10, PL10 |-> PL11, PL11 |-> PL12, PL12 |-> PL13,
     PL13 |-> PL14, PL14 |-> PL15, PL15 |-> PL16, PL16 |-> PL17,
     PL17 |-> PL18, PL18 |-> PL19, PL19 |-> PL20, PL20 |-> PL20,
     PL30 |-> PL31, PL31 |-> PL32, PL33 |-> PL34, PL34 |-> PL35} &
     cl1 = closure1(nxt) & 
     nrs = %x.(x:SIGNAL|card(cl1[{x}])) &
     mx = nrs~[{max(ran(nrs))}]

[2018-06-04 14:09:40,326, T+950175] "ProB Output Logger for instance 1361b11" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] empty_kodkod_union_constraint
[2018-06-04 14:09:40,326, T+950175] "ProB Output Logger for instance 1361b11" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] kodkod fail: nxt = {PL01 |-> PL02,PL02 |-> PL03,PL03 |-> PL04,P...  , reason: integer without upper and lower bound in: {max(ran(nrs))} while extracting used types
Execution time: 0.318373176 seconds


TRUE

Solution:
	cl1 = ∃197∈{(PL01↦PL02),(PL01↦PL03),…,(PL33↦PL35),(PL34↦PL35)}
	mx = {PL01}
	nxt = {(PL01↦PL02),(PL02↦PL03),(PL03↦PL04),(PL04↦PL05),(PL05↦PL06),(PL06↦PL07),(PL07↦PL08),(PL08↦PL09),(PL09↦PL10),(PL10↦PL11),(PL11↦PL12),(PL12↦PL13),(PL13↦PL14),(PL14↦PL15),(PL15↦PL16),(PL16↦PL17),(PL17↦PL18),(PL18↦PL19),(PL19↦PL20),(PL20↦PL20),(PL30↦PL31),(PL31↦PL32),(PL33↦PL34),(PL34↦PL35)}
	nrs = {(PL01↦19),(PL02↦18),(PL03↦17),(PL04↦16),(PL05↦15),(PL06↦14),(PL07↦13),(PL08↦12),(PL09↦11),(PL10↦10),(PL11↦9),(PL12↦8),(PL13↦7),(PL14↦6),(PL15↦5),(PL16↦4),(PL17↦3),(PL18↦2),(PL19↦1),(PL20↦1),(PL21↦0),(PL22↦0),(PL23↦0),(PL24↦0),(PL25↦0),(PL26↦0),(PL27↦0),(PL28↦0),(PL29↦0),(PL30↦2),(PL31↦1),(PL32↦0),(PL33↦2),(PL34↦1),(PL35↦0),(PL36↦0),(PL37↦0),(PL38↦0),(PL39↦0),(PL40↦0)}

In [141]:
:time :solve prob nxt = 
    {PL01 |-> PL02, PL02 |-> PL03, PL03 |-> PL04, PL04 |-> PL05,
     PL05 |-> PL06, PL06 |-> PL07, PL07 |-> PL08, PL08 |-> PL09,
     PL09 |-> PL10, PL10 |-> PL11, PL11 |-> PL12, PL12 |-> PL13,
     PL13 |-> PL14, PL14 |-> PL15, PL15 |-> PL16, PL16 |-> PL17,
     PL17 |-> PL18, PL18 |-> PL19, PL19 |-> PL20, PL20 |-> PL20,
     PL30 |-> PL31, PL31 |-> PL32, PL33 |-> PL34, PL34 |-> PL35} &
     cl1 = closure1(nxt) & 
     nrs = %x.(x:SIGNAL|card(cl1[{x}])) &
     mx = nrs~[{max(ran(nrs))}]

Execution time: 0.046631537 seconds


TRUE

Solution:
	cl1 = ∃197∈{(PL01↦PL02),(PL01↦PL03),…,(PL33↦PL35),(PL34↦PL35)}
	mx = {PL01}
	nxt = {(PL01↦PL02),(PL02↦PL03),(PL03↦PL04),(PL04↦PL05),(PL05↦PL06),(PL06↦PL07),(PL07↦PL08),(PL08↦PL09),(PL09↦PL10),(PL10↦PL11),(PL11↦PL12),(PL12↦PL13),(PL13↦PL14),(PL14↦PL15),(PL15↦PL16),(PL16↦PL17),(PL17↦PL18),(PL18↦PL19),(PL19↦PL20),(PL20↦PL20),(PL30↦PL31),(PL31↦PL32),(PL33↦PL34),(PL34↦PL35)}
	nrs = {(PL01↦19),(PL02↦18),(PL03↦17),(PL04↦16),(PL05↦15),(PL06↦14),(PL07↦13),(PL08↦12),(PL09↦11),(PL10↦10),(PL11↦9),(PL12↦8),(PL13↦7),(PL14↦6),(PL15↦5),(PL16↦4),(PL17↦3),(PL18↦2),(PL19↦1),(PL20↦1),(PL21↦0),(PL22↦0),(PL23↦0),(PL24↦0),(PL25↦0),(PL26↦0),(PL27↦0),(PL28↦0),(PL29↦0),(PL30↦2),(PL31↦1),(PL32↦0),(PL33↦2),(PL34↦1),(PL35↦0),(PL36↦0),(PL37↦0),(PL38↦0),(PL39↦0),(PL40↦0)}

In [5]:
:table {PL01 |-> PL02, PL02 |-> PL03, PL03 |-> PL04, PL04 |-> PL05,
     PL05 |-> PL06, PL06 |-> PL07, PL07 |-> PL08, PL08 |-> PL09,
     PL09 |-> PL10, PL10 |-> PL11, PL11 |-> PL12, PL12 |-> PL13,
     PL13 |-> PL14, PL14 |-> PL15, PL15 |-> PL16, PL16 |-> PL17,
     PL17 |-> PL18, PL18 |-> PL19, PL19 |-> PL20, PL20 |-> PL20,
     PL30 |-> PL31, PL31 |-> PL32, PL33 |-> PL34, PL34 |-> PL35}

|Nr|prj1|prj2|
|---|---|---|
|1|PL01|PL02|
|2|PL02|PL03|
|3|PL03|PL04|
|4|PL04|PL05|
|5|PL05|PL06|
|6|PL06|PL07|
|7|PL07|PL08|
|8|PL08|PL09|
|9|PL09|PL10|
|10|PL10|PL11|
|11|PL11|PL12|
|12|PL12|PL13|
|13|PL13|PL14|
|14|PL14|PL15|
|15|PL15|PL16|
|16|PL16|PL17|
|17|PL17|PL18|
|18|PL18|PL19|
|19|PL19|PL20|
|20|PL20|PL20|
|21|PL30|PL31|
|22|PL31|PL32|
|23|PL33|PL34|
|24|PL34|PL35|


In [142]:
::load
MACHINE SlotSolver_KodkodTest
SETS /* enumerated */
  SLOTS={a1,a2,a3,a4,a5,b1,b2,b3,b4,b5,c1,c2,c3,c4,c5,d1,d2,d3,d4,d5,e1,e2,e3,e4,e5,f1,f2,f3,f4,f5,unknown};
  DAYS={Mon,Tue,Wed,Thu,Fri};
  COURSES={ang,ger,ges,inf,jap,jid,jud,kom,lin,pol,rom,soz};
  TYPES={T_KF,T_EF}
END

[2018-06-04 14:09:40,536, T+950385] "Shell-0" de.prob.cli.PrologProcessProvider.makeProcess(PrologProcessProvider.java:64): [INFO] Starting ProB's Prolog Core. Path is /Users/leuschel/git_root/prob_prolog/probcli.sh
[2018-06-04 14:09:41,873, T+951722] "Shell-0" de.prob.cli.PortPattern.setValue(PortPattern.java:30): [INFO] Server has started and listens on port 63578
[2018-06-04 14:09:41,874, T+951723] "Shell-0" de.prob.cli.InterruptRefPattern.setValue(InterruptRefPattern.java:29): [INFO] Server can receive user interrupts via reference 47994
[2018-06-04 14:09:41,875, T+951724] "ProB Output Logger for instance 1145e2a2" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] -- starting command loop --
[2018-06-04 14:09:41,890, T+951739] "ProB Output Logger for instance 1145e2a2" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Connected: 127.0.0.1


Loaded machine: SlotSolver_KodkodTest : []


In [143]:
:time :solve prob mapping = {
        (lin, 1, T_EF, 102),
        (jap, 1, T_EF, 52),
        (ger, 1, T_KF, 25),
        (inf, 1, T_EF, 43),
        (jap, 2, T_EF, 53),
        (jap, 2, T_EF, 60),
        (rom, 1, T_EF, 132),
        (pol, 1, T_EF, 122),
        (rom, 2, T_EF, 133),
        (ang, 2, T_EF, 2),
        (ang, 2, T_EF, 9),
        (kom, 2, T_EF, 99),
        (rom, 2, T_EF, 125),
        (rom, 1, T_EF, 128),
        (lin, 3, T_EF, 104),
        (inf, 3, T_EF, 41),
        (jid, 2, T_EF, 66),
        (jap, 3, T_EF, 54),
        (rom, 3, T_EF, 138),
        (jap, 1, T_KF, 61),
        (jap, 1, T_KF, 56),
        (ger, 1, T_EF, 29),
        (jap, 2, T_KF, 57),
        (jap, 2, T_KF, 64),
        (jid, 4, T_EF, 68),
        (jud, 1, T_KF, 87),
        (rom, 1, T_KF, 136),
        (rom, 1, T_KF, 150),
        (ang, 2, T_KF, 6),
        (ang, 2, T_KF, 13),
        (ang, 1, T_EF, 16),
        (ang, 1, T_EF, 5),
        (ang, 2, T_KF, 19),
        (ger, 3, T_KF, 26),
        (soz, 2, T_EF, 154),
        (rom, 3, T_KF, 151),
        (rom, 4, T_EF, 127),
        (rom, 3, T_EF, 126),
        (lin, 1, T_EF, 110),
        (ges, 2, T_KF, 32),
        (inf, 2, T_EF, 36),
        (rom, 1, T_EF, 147),
        (jud, 1, T_EF, 86),
        (ang, 1, T_KF, 12),
        (ang, 1, T_KF, 1),
        (lin, 3, T_EF, 112),
        (kom, 1, T_EF, 100),
        (ger, 3, T_EF, 30),
        (jid, 2, T_EF, 81),
        (lin, 4, T_EF, 105),
        (jid, 2, T_EF, 76),
        (jid, 1, T_EF, 69),
        (jud, 3, T_EF, 84),
        (soz, 1, T_EF, 155),
        (jap, 4, T_EF, 51),
        (ger, 1, T_EF, 27),
        (jid, 3, T_EF, 71),
        (ges, 1, T_EF, 33),
        (jud, 1, T_KF, 82),
        (jud, 2, T_KF, 83),
        (pol, 2, T_EF, 121),
        (ang, 2, T_KF, 11),
        (jud, 4, T_KF, 85),
        (rom, 4, T_KF, 146),
        (rom, 3, T_KF, 152),
        (rom, 4, T_KF, 135),
        (jap, 4, T_KF, 55),
        (lin, 2, T_EF, 107),
        (inf, 2, T_EF, 44),
        (jap, 1, T_EF, 59),
        (jap, 2, T_EF, 62),
        (ger, 1, T_KF, 22),
        (ger, 5, T_EF, 24),
        (rom, 2, T_EF, 137),
        (ang, 2, T_EF, 15),
        (ang, 1, T_KF, 20),
        (kom, 2, T_EF, 101),
        (lin, 3, T_EF, 116),
        (lin, 4, T_EF, 113),
        (rom, 3, T_EF, 134),
        (rom, 4, T_EF, 149),
        (rom, 3, T_EF, 148),
        (ang, 3, T_KF, 7),
        (inf, 4, T_EF, 46),
        (rom, 4, T_EF, 131),
        (ang, 4, T_EF, 4),
        (jap, 1, T_KF, 63),
        (ges, 1, T_EF, 31),
        (jid, 3, T_EF, 77),
        (rom, 1, T_KF, 140),
        (rom, 2, T_KF, 141),
        (ang, 1, T_EF, 14),
        (ang, 2, T_KF, 17),
        (ger, 3, T_KF, 28),
        (rom, 3, T_KF, 130),
        (jud, 3, T_KF, 95),
        (rom, 4, T_KF, 143),
        (jap, 1, T_EF, 48),
        (inf, 1, T_EF, 39),
        (jap, 2, T_EF, 49),
        (rom, 1, T_EF, 144),
        (rom, 2, T_EF, 129),
        (ges, 2, T_KF, 34),
        (pol, 1, T_EF, 118),
        (ang, 2, T_EF, 21),
        (ang, 1, T_KF, 10),
        (ang, 1, T_KF, 8),
        (rom, 1, T_EF, 124),
        (ger, 3, T_EF, 23),
        (inf, 3, T_EF, 37),
        (jid, 2, T_EF, 70),
        (jap, 3, T_EF, 50),
        (jid, 2, T_EF, 79),
        (rom, 3, T_EF, 142),
        (rom, 4, T_EF, 139),
        (rom, 3, T_EF, 145),
        (inf, 4, T_EF, 38),
        (soz, 1, T_EF, 153),
        (jap, 4, T_EF, 58),
        (jid, 4, T_EF, 72),
        (jap, 1, T_KF, 52),
        (ger, 1, T_EF, 25),
        (jap, 2, T_KF, 53),
        (jap, 2, T_KF, 60),
        (rom, 1, T_KF, 132),
        (jud, 2, T_KF, 92),
        (rom, 2, T_KF, 133),
        (ang, 2, T_KF, 2),
        (ang, 2, T_KF, 9),
        (pol, 2, T_EF, 123),
        (rom, 2, T_KF, 125),
        (rom, 1, T_KF, 128),
        (jap, 3, T_KF, 54),
        (rom, 3, T_KF, 138),
        (lin, 2, T_EF, 103),
        (jap, 1, T_EF, 61),
        (jap, 1, T_EF, 56),
        (ger, 1, T_KF, 29),
        (jap, 2, T_EF, 57),
        (jap, 2, T_EF, 64),
        (rom, 1, T_EF, 136),
        (rom, 1, T_EF, 150),
        (ang, 2, T_EF, 6),
        (ang, 1, T_KF, 18),
        (ang, 2, T_EF, 19),
        (ang, 1, T_KF, 5),
        (ang, 1, T_KF, 16),
        (inf, 3, T_EF, 45),
        (ger, 3, T_EF, 26),
        (lin, 4, T_EF, 109),
        (rom, 3, T_EF, 151),
        (rom, 4, T_KF, 127),
        (rom, 3, T_KF, 126),
        (ges, 2, T_EF, 32),
        (jid, 4, T_EF, 78),
        (rom, 1, T_KF, 147),
        (jud, 2, T_KF, 94),
        (ang, 1, T_EF, 12),
        (ang, 1, T_EF, 1),
        (ger, 3, T_KF, 30),
        (soz, 2, T_EF, 158),
        (jap, 4, T_KF, 51),
        (lin, 2, T_EF, 111),
        (lin, 1, T_EF, 106),
        (inf, 2, T_EF, 47),
        (inf, 2, T_EF, 40),
        (ger, 1, T_KF, 27),
        (ges, 1, T_KF, 33),
        (jud, 1, T_EF, 82),
        (jud, 2, T_EF, 83),
        (kom, 1, T_EF, 96),
        (pol, 1, T_EF, 120),
        (kom, 2, T_EF, 97),
        (lin, 3, T_EF, 108),
        (lin, 4, T_EF, 117),
        (jid, 2, T_EF, 73),
        (jid, 1, T_EF, 65),
        (rom, 4, T_EF, 146),
        (jud, 3, T_EF, 89),
        (rom, 3, T_EF, 152),
        (ang, 3, T_KF, 3),
        (inf, 4, T_EF, 42),
        (jap, 4, T_EF, 55),
        (rom, 4, T_EF, 135),
        (jap, 1, T_KF, 59),
        (jid, 4, T_EF, 75),
        (jap, 2, T_KF, 62),
        (ger, 1, T_EF, 22),
        (jid, 3, T_EF, 67),
        (ger, 5, T_KF, 24),
        (jid, 3, T_EF, 74),
        (jud, 1, T_KF, 91),
        (jud, 2, T_KF, 88),
        (rom, 2, T_KF, 137),
        (ang, 2, T_KF, 15),
        (soz, 2, T_EF, 156),
        (rom, 3, T_KF, 134),
        (rom, 4, T_KF, 149),
        (rom, 3, T_KF, 148),
        (ang, 3, T_EF, 7),
        (rom, 4, T_KF, 131),
        (lin, 1, T_EF, 114),
        (ang, 4, T_KF, 4),
        (jap, 1, T_EF, 63),
        (lin, 2, T_EF, 115),
        (ges, 1, T_KF, 31),
        (inf, 1, T_EF, 35),
        (jud, 1, T_EF, 90),
        (rom, 1, T_EF, 140),
        (rom, 2, T_EF, 141),
        (ang, 2, T_EF, 17),
        (ang, 1, T_KF, 14),
        (kom, 1, T_EF, 98),
        (ger, 3, T_EF, 28),
        (rom, 3, T_EF, 130),
        (rom, 4, T_EF, 143),
        (soz, 1, T_EF, 157),
        (jap, 1, T_KF, 48),
        (rom, 1, T_KF, 144),
        (jap, 2, T_KF, 49),
        (jid, 3, T_EF, 80),
        (jud, 1, T_KF, 93),
        (rom, 2, T_KF, 129),
        (ges, 2, T_EF, 34),
        (pol, 2, T_EF, 119),
        (ang, 1, T_EF, 10),
        (ang, 2, T_KF, 21),
        (ang, 1, T_EF, 8),
        (rom, 1, T_KF, 124),
        (ger, 3, T_KF, 23),
        (jap, 3, T_KF, 50),
        (rom, 3, T_KF, 142),
        (rom, 4, T_KF, 139),
        (rom, 3, T_KF, 145),
        (jap, 4, T_KF, 58)
    } &
    cs = {c|#(sem,typ,nr).( (c,sem,typ,nr):mapping & nr:100..150)}

Execution time: 0.353512947 seconds


TRUE

Solution:
	cs = {kom,lin,pol,rom}
	mapping = ∃235∈{(((ang↦1)↦T_KF)↦1),(((ang↦1)↦T_KF)↦5),…,(((soz↦2)↦T_EF)↦156),(((soz↦2)↦T_EF)↦158)}